# Assignment \#2: River Basin Sensitivity Vulnerability and Robustness Analysis

This assignment builds on lab exercise #4.2 where you selected the policies with the best performance across each of the six Nile basin's objectives, as well as two compromise solutions.  For this assignment, we focus on the external factors (the X part of the XLRM diagram), these are factors which are not under our control, yet, they may be influential on the problem's objectives. To this end, you are expected to apply [**exploratory modelling and analysis**](https://www.jstor.org/stable/171847). 

In [ ]:
# Importing required packages
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from tqdm import tqdm
from IPython.display import Image

# We need to give permission for connecting to our drive files
from google.colab import drive
drive.mount('/content/drive')

# Copying the files from our drive folder. The folder name after ../MyDrive/ must match
!cp /content/drive/MyDrive/water_systems -r /content

from water_systems.model_nile import ModelNile
from water_systems import plotter

Mounted at /content/drive


## Defining the states-of-the-world

The idea is to test the Nile sytem model outcomes under a suite of  states-of-the-world (SOWs), this is a fundamental concept in decision analysis which indicates the decision-maker's and modeler's views on uncertainty about the world. In the Nile case, we identify seven uncertain factors that may impact the outcomes of interest.  For each variable, we assume a lower and upper bound described in the following table. 

Uncertain Variable| Baseline value | Range|
------------------|----------------|-------------------------|
Annual demand growth rate| 0.0212| 0.01 - 0.03|
Blue Nile mean inflow coefficient| 1 |0.75 - 1.25|
White Nile mean inflow coefficient| 1 |0.75 - 1.25|
Atbara mean inflow coefficient| 1 |0.75 - 1.25|
Blue Nile deviation coefficient| 1 |0.5 - 1.5|
White Nile deviation coefficient| 1 |0.5 - 1.5|
Atbara deviation coefficient| 1 |0.5 - 1.5|

## Experimental design

In order to sample from among all the uncertain variables simultaneously, we will use [Latin Hypercube Sampling (LHS)](https://en.wikipedia.org/wiki/Latin_hypercube_sampling), which is a multi-dimensional sampling technique. 
Once we have a candidate set of samples, we will run a simulation for each policy and scenario combination.  We will use the [Exploratory Modelling and Analysis (EMA) Workbench](https://emaworkbench.readthedocs.io/en/latest/), run the code below to install and import the required python classes from the ema_workbench. 

In [ ]:
!pip install ema_workbench
from ema_workbench import RealParameter, ScalarOutcome, Model, Policy
from ema_workbench import MultiprocessingEvaluator, ema_logging

## Connecting the EMA Workbench with the Nile Model

The code below connects the Nile model with the EMA workbench.  We first specify the problem, the uncertainties, the policy levers, and the outcomes of interest.

In [ ]:
# Generating ModelNile object and connecting it to the EMA Workbench:
nile_model = ModelNile()
em_model = Model("NileProblem", function=nile_model)

# Specifying the uncertainties:
em_model.uncertainties = [
    RealParameter("yearly_demand_growth_rate", 0.01, 0.03),
    RealParameter("blue_nile_mean_coef", 0.75, 1.25),
    RealParameter("white_nile_mean_coef", 0.75, 1.25),
    RealParameter("atbara_mean_coef", 0.75, 1.25),
    RealParameter("blue_nile_dev_coef", 0.5, 1.5),
    RealParameter("white_nile_dev_coef", 0.5, 1.5),
    RealParameter("atbara_dev_coef", 0.5, 1.5),
]

# Now the levers and lever ranges. Note that these correspond to RBF parameters!
parameter_count = nile_model.overarching_policy.get_total_parameter_count()
n_inputs = nile_model.overarching_policy.functions["release"].n_inputs
n_outputs = nile_model.overarching_policy.functions["release"].n_outputs
RBF_count = nile_model.overarching_policy.functions["release"].RBF_count
p_per_RBF = 2 * n_inputs + n_outputs # weights- outputs for each input > center and weights.

lever_list = list()
for i in range(parameter_count):
    modulus = (i - n_outputs) % p_per_RBF #parameters per RBF
    if (
        (i >= n_outputs)
        and (modulus < (p_per_RBF - n_outputs))
        and (modulus % 2 == 0)
    ):  # centers:
        lever_list.append(RealParameter(f"v{i}", -1, 1))
    else:  # linear parameters for each release, radii and weights of RBFs:
        lever_list.append(RealParameter(f"v{i}", 0, 1))

em_model.levers = lever_list

# Specify outcomes of interest:
em_model.outcomes = [
    ScalarOutcome("egypt_irr", ScalarOutcome.MINIMIZE),
    ScalarOutcome("egypt_90", ScalarOutcome.MINIMIZE),
    ScalarOutcome("egypt_low_had", ScalarOutcome.MINIMIZE),
    ScalarOutcome("sudan_irr", ScalarOutcome.MINIMIZE),
    ScalarOutcome("sudan_90", ScalarOutcome.MINIMIZE),
    ScalarOutcome("ethiopia_hydro", ScalarOutcome.MAXIMIZE),
]

### Part 1: Running Simulations with each Policy and Scenario Combination

To guarantee good coverage of the uncertainty space, we need to run our simulation under a large set of scenarios.  Your task is to **run 1000 scenarios with at most 5 policies** that you selected in the lab session. Keep in mind that each experiment takes around 2 seconds. So, if you are running 1000*5=5000 combinations, the expected runtime is approximately 10000 seconds (almost 3 hours). You can use the logging function to collect information about the time to completion during the run.

<br>

The function *evaluator.perform_experiments* returns: 
1. A dataframe that stores the experiment details (experiments)
2. A dictionary of arrays of objective values (outcomes)

<br>

**Important Note:** Export your experiment outputs to avoid loosing them when the runtime is restarted.

Tip: Start small, that is, perform a few trial runs with a small number of scenarios and policies, make sure that everything runs as you expect, and that you are able to keep track of the simulation outcomes before you launch a larger experiment. 

In [ ]:
# Number of scenarios to be sampled from the uncertainty ranges
n_scenarios = 200

# Read the policies we prepared during the lab session
policy_df = pd.read_csv("water_systems/policies_for_exploration.csv")
my_policies = [
    Policy(policy_df.loc[i, "name"], **(policy_df.iloc[i, :-1].to_dict()))
    for i in policy_df.index
]

# Turning on logging to get informed during the run
ema_logging.log_to_stderr(ema_logging.INFO)

with MultiprocessingEvaluator(em_model) as evaluator:
    experiments, outcomes = evaluator.perform_experiments(
        n_scenarios, # Integer input means EMA will sample this many scenarios for us
        my_policies # Policy objects as input means it will run the sampled scenarios with each of the policies
    )

### Part 2: Global Sensitivity Analysis

In this part, we will examine the sensitivity of the outcomes of interest to  uncertain conditions and to the policy selection. Building on the outcomes generated in Part 1, we will infer variable significance, that is, which factor matters the most influence in our outcomes of interest.  There are a number of machine learning techniques to generate these insights. EMA Workbench supports [feature scoring](https://emaworkbench.readthedocs.io/en/latest/indepth_tutorial/open-exploration.html#feature-scoring) which is based on fitting multiple regression/classification trees for the relationship between inputs and outputs. **Calculate the feature scores of all uncertain variables as well as the selection of policy on every outcome of interest**. This corresponds to 8 feature score inputs (7 uncertain veriables + 1 column for the policy selection) and 6 feature score outputs (1 for each objective). Think about the arguments you use while calling the functions that calculate feature scores. You can refer to the [GitHub page](https://github.com/quaquel/EMAworkbench/blob/master/ema_workbench/analysis/feature_scoring.py) which contains the feature scoring source code.  **Reflect** on your findings.  Which factors or combination of factors are the most influential for our analysis?  What are the implications for the Nile system?  What are the potential risks and opportunities for the three countries involved?


### Part 3: Factors contributing to the river basin's goals success or failure. 

Feature scoring tells us how important each variable is in explaining the variability of an outcome of interest. Nonetheless, it does not indicate the direction of influence. In this part, we will investigate how the regions of the uncertainty space are linked to the objective performance. A useful visual method to do so is [dimensional stacking](https://emaworkbench.readthedocs.io/en/latest/indepth_tutorial/open-exploration.html#dimensional-stacking). To generate a dimensional stacking graph, first we need to specify a performance threshold for the objectives, we then classify the outcome values that are lower or higher than our specified threshold. We can then show these results graphically by color coding the regions of the uncertainty space based on the density of classified observations in each region.  Investigate which combination of uncertainties would contribute to the system's vulnerability, and identify opportunities for the outcomes of interest. **Reflect** on what your findings mean for the Nile system and the risks and opportunities for the three countries involved.

### Part 4. Robustness of Policies

An essential tool for decision making under deeply uncertain conditions is [robust decision making (RDM)](https://link.springer.com/chapter/10.1007/978-3-030-05252-2_2). The main philosophy behind the RDM approach is to stress-test the performance of policies under many alternative states of the world and identify the ones that lead to robust performance across all scenarios. Several [metrics to quantify robustness](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2017EF000649) are available.  Calculate the robustness score for each policy with respect to a metric you choose and show your results graphically.  Is it possible to recommend a policy based on this analysis? What are the underlying assumptions for the analysis you conducted and for your recommendations to be valid?